In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [8]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import OllamaEmbeddings, ChatOllama
import os
import re
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

import pandas as pd

load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'
OLLAMA_EMBEDDING_MODEL = 'bge-m3'

PINECONE_INDEX_NAME_LAW = 'law'
PINECONE_INDEX_NAME_RULE = 'rule'
PINECONE_INDEX_NAME_CASE = 'case'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

In [57]:
# ==============
# 상용님 카테고리
# ==============

def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2
        },
        # [신설] 9. 임차권 승계 및 가족 (제9조 타겟팅)
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        }
    }


def categorize_content(content, top_k=None):
    '''
    내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters : 
        - content : 분류할 텍스트 내용
        - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
        
    Returns :
        - 카테고리 리스트(점수 높은 순)
    '''
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            # if keyword in content: # content에 keyword가 포함되어있는지 여부
                # score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0 :
            category_scores[category] = score
    # print(category_scores)
    
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x : x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    
    # 매칭되는 카테고리가 없으면 '기타' 반환
    if not all_categories:
        all_categories = ['기타']
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    return all_categories


# =====================
# 참조조항을 추출하는 함수
# =====================

def extract_articles_from_docs(documents):
    '''검색된 문서들(documents)에서 aritcle 정보를 추출하여 리스트로 변환
    Returns : 중복이 제거된 조항 리스트 ex : ['제4조', '제16조']'''
    articles = []
    for doc in documents:
        article = doc.metadata.get('article', '조항없음')
        
        # 「」제거. 문자를 list로 분리
        if article != '조항없음':
            article = article.replace('「','').replace('」','')
            article_list = article.split(', ')
            articles.extend(article_list)
            
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    final_articles = ['제'+article for article in unique_articles] 
    return '주택임대차보호법' + ','.join(final_articles)


# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query:str, documents:list, top_k:int=4) -> list:
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    docs_text = [doc.metadata.get('title','') + ' ' + doc.page_content for doc in documents]
    
    # cohere API 호출
    results = co.rerank(model='rerank-multilingual-v3.0', # 한국어 지원 모델
                        query=query,
                        documents=docs_text,
                        top_n=top_k)
    
    # 재조정된 문서 반환
    idxs = [r.index for r in results.results] # 재조정된 문서 index
    reranked_docs = [documents[i] for i in idxs]
    return reranked_docs

In [63]:
def get_token_usage(llm, llm_response):
    """
    langchain_openai.ChatOpenAI 응답에서 토큰 사용량 추출.
    - 우선: llm_response.usage_metadata (네 로그에서 실제 존재)
    - 차선: llm_response.response_metadata["token_usage"]
    - 그 외: None
    """
    # OpenAI가 아니면 None (Ollama 등)
    if llm.__class__.__module__ != "langchain_openai.chat_models.base":
        return None

    # 1) ✅ 네 환경에서 확실히 존재하는 usage_metadata
    usage_meta = getattr(llm_response, "usage_metadata", None)
    if isinstance(usage_meta, dict) and usage_meta:
        return {
            "input_tokens": usage_meta.get("input_tokens"),
            "output_tokens": usage_meta.get("output_tokens"),
            "total_tokens": usage_meta.get("total_tokens"),
        }

    # 2) 일부 케이스: response_metadata 안 token_usage
    meta = getattr(llm_response, "response_metadata", None) or {}
    token_usage = meta.get("token_usage")
    if isinstance(token_usage, dict) and token_usage:
        # token_usage는 보통 prompt_tokens / completion_tokens 형태일 수 있음
        prompt = token_usage.get("prompt_tokens") or token_usage.get("input_tokens")
        completion = token_usage.get("completion_tokens") or token_usage.get("output_tokens")
        total = token_usage.get("total_tokens")
        if total is None and prompt is not None and completion is not None:
            total = prompt + completion
        return {
            "input_tokens": prompt,
            "output_tokens": completion,
            "total_tokens": total,
        }

    return None



import re

def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_by_priority(
    law_db: PineconeVectorStore,
    embedding: UpstageEmbeddings,
    query_for_embed: str,
    categories: list,
    k_total: int = 4,
    k_each: int = 4,
    priorities: list = [1,2,3,4,5,6,7],
) -> list:
    """
    law 인덱스에서 priority=1부터 순차적으로 확인하며 문서를 누적 수집.
    - embedding은 1회만 생성
    - priority가 낮을수록(1이 가장 중요) 우선적으로 채움
    """
    # Upstage에 넣는 query는 반드시 정제된 값을 사용
    clean_q = safe_embedding_query(query_for_embed, query_for_embed)
    qvec = embedding.embed_query(clean_q)

    collected = []
    seen = set()

    for p in priorities:
        flt = {
            "category": {"$in": categories},
            "priority": {"$eq": p},
        }

        docs = law_db.similarity_search_by_vector(
            qvec,
            k=k_each,
            filter=flt
        )

        for d in docs:
            # 중복 제거(간단 key)
            key = (d.metadata.get("id"), d.metadata.get("source"), d.page_content[:120])
            if key in seen:
                continue
            seen.add(key)
            d.metadata.setdefault("priority", p)
            collected.append(d)

        if len(collected) >= k_total:
            break

    return collected[:k_total]


# 1. 비용계산

In [49]:
OPENAI_PRICES = {
    "gpt-4o-mini": {
        "input": 0.15 / 1000000,
        "output": 0.60 / 1000000,
    },
    "gpt-4.1-mini": {
        "input": 0.40 / 1000000,
        "output": 1.60 / 1000000,
    },
}

def calculate_openai_cost(model_name: str, token_usage: dict) -> float | None:
    if token_usage is None:
        return None
    price = OPENAI_PRICES.get(model_name)
    if not price:
        return None

    return (
        token_usage["input_tokens"]  * price["input"] +
        token_usage["output_tokens"] * price["output"]
    )


# 2. LLM

## index law

In [71]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4, top_k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    # llm = ChatOpenAI(model='gpt-4o-mini')
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOllama(model="bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh", temperature=0.1)
    # llm = ChatOllama(model="mervinpraison/llama3.1-instruct:8b", temperature=0.1)
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)
    ol_embedding = OllamaEmbeddings(model=OLLAMA_EMBEDDING_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    case_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 보증금 반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # [행위 - 거주/계약]
        "계약서 -> 임대차계약증서", "특약 -> 특약사항",
        "이사 -> 주택의인도", "전입 -> 주민등록/대항력",
        "살고 -> 점유", "거주 -> 점유",
        "나가라 -> 계약갱신거절/명도", "나가라고 -> 계약갱신거절/명도", "비워 -> 명도", "쫓겨 -> 명도/대항력",
        "방 빼 -> 계약해지", "나갈게 -> 계약해지", "해지 -> 계약해지",
        "연장 -> 계약갱신", "더 살 -> 계약갱신", "재계약 -> 계약갱신",
        "수리 -> 수선의무/필요비", "고쳐 -> 수선의무", "물 새 -> 누수", "곰팡이 -> 하자",
        "청소비 -> 원상회복비용", "원상복구 -> 원상회복"
    ]

    prompt = ChatPromptTemplate.from_template(f'''아래 질문에서 단어만 사전을 기준으로 치환하세요.문장 구조, 시제, 의문형 여부는 변경하지 마세요
    사전 : {keyword_dict}
    질문 : {{question}}
    출력은 질문 문장만 하세요''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    # 4. 질문을 표준화
    normalized_query = keyword_chain.invoke({'question':query}) # 출력함

    # 5. 관련문서 검색
    retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    # 관련문서  rerank
    embed_query = safe_embedding_query(normalized_query, query)
    categories = categorize_content(query)
    retrieved_docs = retrieve_law_docs_by_priority(
        law_db=law_database,
        embedding=up_embedding,
        query_for_embed=embed_query,
        categories=categories,
        k_total=k,         # 최종 문서 수는 기존 k를 따름
        k_each=max(k, 4),  # 각 priority에서 몇 개씩 볼지(넉넉히)
        priorities=[1,2,3,4,5,6,7]
    )
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k) # 최종 documents
    referenced_articles = extract_articles_from_docs(reranked_docs) # 참조조항 출력

    # 프롬프트 템플릿 (위에것 그대로)을 이용한 rag체인 구성
    # template = f'당신은 최고의 한국 주택임대차보호법 전문가입니다. 다음 문맥을 참고하여 질문에 답하세요. 답을 모르면 모른다고 답하세요. 최대 3문장으로 간결하게 답변하세요. 질문 : {{query}} / 문맥 : {{context}} / 답변 :'
    template = f"""
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
    2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
    3. **답변 구조**:
       - 결론부터 명확히(가능/불가능) 제시
       - 📖 법적 근거(제N조) 상세 설명
       - 사용자 상황에 맞는 조언
    4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
    5. **❗ 면책**: 법적 효력이 없음을 밝히세요.

    [참고 법령]
    {{context}}
    질문 : {{query}}
    """
    prompt = ChatPromptTemplate.from_template(template)
    # prompt_chain = prompt | llm | StrOutputParser()
    # result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  # 'query':query})
    prompt_chain = prompt | llm
    llm_response = prompt_chain.invoke({'context':format_documents(reranked_docs),
                                        'query':query})
    result = llm_response.content
    
    
    print('원본 질문 :', query)
    print('표준화된 질문 :', normalized_query)
    print('='*90)
    print('\n✅', result)
    # print('\n📌 참조사항 :', referenced_articles)
    # print('\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗')
    
    token_usage = get_token_usage(llm, llm_response)
    cost = calculate_openai_cost("gpt-4.1-mini", token_usage)
    
    print("\n📊 토큰 사용량")
    if token_usage is None:
        print("- Input tokens  : None")
        print("- Output tokens : None")
        print("- Total tokens  : None")
    else:
        print(f"- Input tokens  : {token_usage['input_tokens']}")
        print(f"- Output tokens : {token_usage['output_tokens']}")
        print(f"- Total tokens  : {token_usage['total_tokens']}")
        
    print("\n💰 추정 비용")
    print(cost if cost is not None else "None")
    
#     print("LAW picked priorities:", [d.metadata.get("priority") for d in retrieved_docs])

    print("LAW picked priorities:", [d.metadata.get("priority") for d in retrieved_docs])
    for i, d in enumerate(reranked_docs):
        title = d.metadata.get("title") or d.metadata.get("source") or ""
        snippet = d.page_content[:120].replace("\n"," ")
        print(i, d.metadata.get("priority"), title, "|", snippet)

### 모델 비교

In [72]:
%%time
# 사용예시 : exaone3.5:2.4b
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('계약이 1달 남은 시점에서 집주인이 갑자기 나가래요')

원본 질문 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요
표준화된 질문 : 계약이 1달 남은 시점에서 임차인이 갑자기 나가래요

✅ ### 결론
**불가능**: 현재 계약 기간이 1달 남아 있는 상황에서 집주인이 갑자기 계약 해지를 요구하는 것은 법적으로 제한적입니다. 특히, 대한민국의 주택임대차보호법에 따르면 임차인에게 불리한 계약 해지 요구는 강행규정 위반으로 간주되어 효력이 없을 가능성이 큽니다.

### 법적 근거 (제10조 강행규정)
**제10조(강행규정)**: 이 법에 위반된 약정으로서 임차인에게 불리한 것은 그 효력이 없다.

- **상세 설명**:
  - **강행규정**: 주택임대차보호법 제10조는 임차인에게 불리한 계약 조건이나 해지 요구를 법적으로 강제할 수 없음을 명시하고 있습니다. 즉, 집주인이 임차인에게 불리한 조건으로 계약 해지를 강요하는 것은 법적으로 허용되지 않습니다.
  - **적용 사례**: 계약 기간 중인 임차인에게 갑작스럽고 부당한 해지 요구는 법적 보호를 받을 수 있으며, 이러한 요구가 있을 경우 임차인은 법적 조치를 취할 수 있습니다.

### 사용자 상황에 맞는 조언
1. **즉시 대응**:
   - **연락**: 즉시 집주인에게 상황을 설명하고 합리적인 해결책을 모색하십시오. 예를 들어, 계약 조건 재확인, 중도 해지 가능성 검토 등을 제안할 수 있습니다.
   
2. **법률 상담**:
   - **전문가 상담**: 부동산 법률 전문가나 변호사와 상담하여 구체적인 법적 권리와 대응 방안을 확인하세요. 특히, 계약 해지의 적법성과 관련된 증거 수집 및 법적 절차에 대해 조언을 구하는 것이 중요합니다.

3. **증거 수집**:
   - **문서화**: 계약 조건, 이전 통신 기록, 계약서 사본 등 모든 관련 문서를 체계적으로 보관하세요. 이는 향후 법적 분쟁 시 중요한 증거가 될 수 있습니다.

### 위험 경고 및 면책
**위험 경고**:
- **강행규정 위반**: 집주인이 강행규정을 무시하고 임차인에게 불

In [67]:
%%time
# 사용예시 : gpt-4o-mini
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('계약이 1달 남은 시점에서 집주인이 갑자기 나가래요')

원본 질문 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요
표준화된 질문 : 계약이 1달 남은 시점에서 임대인이 갑자기 나가래요

✅ 결론: **불가능**입니다.

📖 법적 근거:
주택임대차보호법 제10조(강행규정)에서는 임대차 계약에 관한 약정이 임차인에게 불리하게 되어서는 안 된다고 규정하고 있습니다. 즉, 임대인이 임대차 계약의 종료를 일방적으로 통보할 수 있는 권리는 없습니다. 임대차 계약이 만료되기 전에는 임차인이 계약을 유지할 수 있는 권리가 있으며, 집주인은 계약이 만료될 때까지 임차인을 내보낼 수 없습니다.

사용자 상황에 맞는 조언:
계약이 만료되기 1개월 남은 시점에서 집주인이 나가라고 요구하는 것은 법적으로 무효입니다. 만약 이와 같은 상황이 계속된다면, 집주인과의 대화를 통해 계약 조건을 다시 설명하고, 필요할 경우 임대차 계약의 내용을 서면으로 정리하여 증거로 확보하는 것이 중요합니다. 

🚨 위험 경고:
임대차 계약에서 불리한 약정이나 특약이 포함되어 있다면 이는 법적으로 무효가 될 수 있습니다(제10조). 따라서 계약서의 내용을 반드시 점검하시고, 불리한 조항이 있다면 그것을 지적하고 수정하도록 요구하시기 바랍니다.

❗ 면책:
이 답변은 법적 효력이 없으며, 구체적인 법률 문제에 대해서는 반드시 전문 변호사와 상담하는 것을 권장합니다.

📊 토큰 사용량
- Input tokens  : 447
- Output tokens : 324
- Total tokens  : 771

💰 추정 비용
0.00026145
LAW picked priorities: [1.0, 1.0, 1.0, 2.0]
CPU times: total: 156 ms
Wall time: 10.8 s


In [69]:
%%time
# 사용예시 : gpt-4.1-mini
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('계약이 1달 남은 시점에서 집주인이 갑자기 나가래요')

원본 질문 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요
표준화된 질문 : 계약이 1달 남은 시점에서 임대인이 갑자기 계약갱신거절/명도래요

✅ 결론:  
계약 종료 1달 전에 집주인이 일방적으로 계약 해지를 통보하고 퇴거 요구하는 것은 **원칙적으로 불가능**합니다. 특히 임대차계약이 종료되기 전 임차인의 거주권을 침해하는 행위로 간주됩니다.

📖 법적 근거:  
- 「주택임대차보호법」 제10조: 본 법에 위반된 약정으로서 임차인에게 불리한 사항은 효력이 없습니다.  
- 계약 기간이 남아 있을 경우, 임대인은 정당한 사유 없이 임차인을 내보낼 수 없습니다.  
- 임대차 계약 종료 시 임차인은 계약 만료일까지 거주할 권리가 보장됩니다. 집주인의 일방적 퇴거 요구는 명도의 사전 법적 절차 없이 임차인의 대항력을 무시하는 행위로 문제가 될 수 있습니다.  

조언:  
- 계약 종료일까지 안정적으로 거주할 권리가 있으므로 권리를 침해당했다고 생각되면 계약서와 관련 증거(월세 납입 영수증 등)를 준비하세요.  
- 임대인의 무리한 퇴거 요구가 지속되거나 강압적인 경우, 주택임대차보호법 및 민법상 명도 소송을 통해 권리 보호 조치를 고려해야 합니다.  
- 임대차 종료 이후 상황을 대비해 집주인과 명확한 합의와 퇴거 일정을 문서화하는 것을 권장합니다.  
- 불리한 내용의 특약이 있더라도 제10조에 따라 법적 효력은 없으므로 강압에 굴복하지 마세요.  

🚨 위험 경고:  
집주인이 임차인의 동의 없이 임대차 기간 중 일방적으로 퇴거를 요구하거나, 법적 절차를 거치지 않은 강제 명도 시도는 불법이며 형사적, 민사적 책임으로 이어질 수 있으니 각별히 주의하세요.  

❗ 면책:  
본 답변은 일반적인 법률 정보 제공을 목적으로 하며, 구체적 사실관계에 따른 법률적 판단을 대신하지 않습니다. 필요시 전문 변호사 상담을 권장합니다.

📊 토큰 사용량
- Input tokens  : 447
- Output tokens : 491
- Total tokens  : 

## index 추가

In [29]:
from typing import Dict, List, Tuple, Optional

# =======================================
# 유틸: 문서 중복 제거 (내용+메타 기반 간단 키)
# =======================================

def dedupe_docs(docs: List[Document]) -> List[Document]:
    seen = set()
    out = []
    for d in docs:
        key = (
            d.metadata.get("id") or d.metadata.get("source") or "",
            d.metadata.get("article") or "",
            d.metadata.get("priority") or "",
            d.page_content[:200],  # 너무 길면 부담, 앞부분만
        )
        if key in seen:
            continue
        seen.add(key)
        out.append(d)
    return out


# =======================================
# 유틸: priority별로 여러 벡터스토어에서 검색
# =======================================

def retrieve_by_priority(
    query: str,
    vectorstores: Dict[str, "PineconeVectorStore"],  # {"law": law_db, "rule": rule_db, "case": case_db}
    categories: List[str],
    priorities: List[int] = [1,2,3,4,5,6,7],
    k_per_store: int = 4,
    use_priority_filter: bool = True,
) -> Dict[int, List[Document]]:
    """
    반환: {priority: [docs...]}  (law/rule/case 통합)
    """
    results: Dict[int, List[Document]] = {p: [] for p in priorities}

    # Pinecone 메타 필터 
    # priority 필드는 int
    for p in priorities:
        for store_name, store in vectorstores.items():
            # 필터 구성
            base_filter = {"category": {"$in": categories}}
            if use_priority_filter:
                base_filter["priority"] = {"$eq": p}

            retriever = store.as_retriever(search_kwargs={"k": k_per_store, "filter": base_filter})
            docs = retriever.invoke(query)

            # 출처/인덱스 정보 태깅(나중에 출력/디버깅에 유용)
            for d in docs:
                d.metadata["index"] = store_name
                # priority가 문서 메타에 없다면 p로 보정
                d.metadata.setdefault("priority", p)

            results[p].extend(docs)

        results[p] = dedupe_docs(results[p])

    return results


# ===================================================
# 메인: ask_with_reference (멀티 인덱스 + priority 우선)
# ===================================================

def ask_with_reference(query: str, k: int = 4):
    """
    query, 표준화된 query, 답변, (priority별) 참조사항 출력
    - 답변 컨텍스트: priority=1 문서만 우선 사용
    - 추가참고: priority=2~7 문서가 있으면 함께 출력
    """

    # 1) llm, embedding
    # llm = ChatOpenAI(model=OPENAI_LLM_MODEL)
    # llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOllama(model="bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh", temperature=0.1)
    llm = ChatOllama(model="mervinpraison/llama3.1-instruct:8b", temperature=0.1)
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    # 2) vector DB 연결 (3개 다)
    law_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_LAW)
    rule_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_RULE)
    case_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_CASE)

    vectorstores = {
        "law": law_database,
        "rule": rule_database,
        "case": case_database,
    }

    # 3) keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 보증금 반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # [행위 - 거주/계약]
        "계약서 -> 임대차계약증서", "특약 -> 특약사항",
        "이사 -> 주택의인도", "전입 -> 주민등록/대항력",
        "살고 -> 점유", "거주 -> 점유",
        "나가라 -> 계약갱신거절/명도", "나가라고 -> 계약갱신거절/명도", "비워 -> 명도", "쫓겨 -> 명도/대항력",
        "방 빼 -> 계약해지", "나갈게 -> 계약해지", "해지 -> 계약해지",
        "연장 -> 계약갱신", "더 살 -> 계약갱신", "재계약 -> 계약갱신",
        "수리 -> 수선의무/필요비", "고쳐 -> 수선의무", "물 새 -> 누수", "곰팡이 -> 하자",
        "청소비 -> 원상회복비용", "원상복구 -> 원상회복"
    ]

    prompt = ChatPromptTemplate.from_template(f"""
아래 질문에서 단어만 사전을 기준으로 치환하세요.
문장 구조, 시제, 의문형 여부는 변경하지 마세요.
사전 : {keyword_dict}
질문 : {{question}}
출력은 질문 문장만 하세요.
""")

    keyword_chain = prompt | llm | StrOutputParser()

    # 4) 질문 표준화
    normalized_query = keyword_chain.invoke({"question": query})

    # 5) 카테고리 분류 (원본/표준화 중 택1)
    categories = categorize_content(query)

    # 6) priority 우선 검색: 모든 인덱스(law/rule/case)를 priority=1부터 검색
    #    - k_per_store를 키우면 priority=1 근거가 더 잘 잡힘
    grouped_docs = retrieve_by_priority(
        query=normalized_query,
        vectorstores=vectorstores,
        categories=categories,
        priorities=[1,2,3,4,5,6,7],
        k_per_store=max(2, k // 2) if k else 4,  # 대충 균형
        use_priority_filter=True,
    )

    # 7) 답변 컨텍스트는 priority=1만 우선 사용 (없으면 2,3...로 폴백)
    context_docs = grouped_docs.get(1, [])
    used_priority = 1
    if len(context_docs) == 0:
        for p in [2,3,4,5,6,7]:
            if grouped_docs.get(p):
                context_docs = grouped_docs[p]
                used_priority = p
                break

    # (선택) 답변 컨텍스트 길이 제한: 너무 많으면 상위 k개만
    context_docs = context_docs[:k]

    # 8) 참조조항 추출 (priority별로도 뽑고 싶으면 반복하면 됨)
    referenced_articles_main = extract_articles_from_docs(context_docs)

    # 9) RAG 프롬프트
    template = f"""
당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

[지시사항]
1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
3. **답변 구조**:
   - 결론부터 명확히(가능/불가능) 제시
   - 📖 법적 근거(제N조) 상세 설명
   - 사용자 상황에 맞는 조언
4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
5. **❗ 면책**: 법적 효력이 없음을 밝히세요.

[참고 법령] (priority={used_priority} 우선 근거)
{{context}}

질문 : {{query}}
답변 :
"""
    rag_prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = rag_prompt | llm | StrOutputParser()

    result = prompt_chain.invoke({
        "context": format_documents(context_docs),
        "query": query
    })

    # 10) 출력: priority=1 답변 + 추가근거(priority 2~7)
    print("원본 질문 :", query)
    print("표준화된 질문 :", normalized_query)
    print("=" * 90)
    print("\n✅", result)

#     # 메인 근거
#     print("\n📌 메인 근거(priority=%d) :" % used_priority, referenced_articles_main)

#     # 추가 근거(부가적으로 같이 출력)
#     extra_refs = {}
#     for p in [1,2,3,4,5,6,7]:
#         if p == used_priority:
#             continue
#         if grouped_docs.get(p):
#             extra_refs[p] = extract_articles_from_docs(grouped_docs[p])

#     if extra_refs:
#         print("\n🧩 추가 참고(부가 근거) :")
#         for p in sorted(extra_refs.keys()):
#             print(f"  - priority={p} :", extra_refs[p])

    # print("\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗")


### 모델 비교

In [16]:
%%time
# 사용예시 : exaone3.5:2.4b

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 질문 : 벽지 손상 시 임차인이 전체 수리 비용을 부담하고 계약을 해지한다고 명시되어 있나요?

✅ 답변 : ### 답변 요약
**결론**: 불가능 (법률 위반 가능성)

**법적 근거**:
- **주택임대차보호법 제6조 (임대차의 해지 및 해지의 효력)** 및 **제10조 (임대차계약의 해지 사유)**
  - **제6조**: 임대차계약의 해지 사유 중 하나로 '임대인의 중대한 과실'이 명시되어 있습니다. 벽지 손상과 같은 임차인의 책임 범위를 넘어서는 임대인의 과실이 인정될 경우, 임차인은 계약 해지를 주장할 수 있습니다.
  - **제10조**: 계약 해지 시 임대인은 임차인에게 적절한 보상을 제공해야 합니다. 단순히 임차인이 모든 수리 비용을 부담하고 계약을 종료하는 것은 임대인의 의무 이행 범위를 벗어난 경우가 많습니다.

**상세 설명**:
- **임대인의 책임**: 벽지 손상과 같은 문제는 일반적으로 임대인의 관리 부실이나 부주의로 인한 경우가 많습니다. 만약 임대인이 이러한 유지보수 책임을 충분히 이행하지 않았다면, 임차인은 계약 해지를 주장할 수 있습니다.
- **임차인의 권리**: 임차인은 계약 조건에 명시된 책임 범위 내에서만 수리 비용을 부담해야 합니다. 만약 계약서에 명시된 내용이 법률상 허용되는 범위를 벗어난다면, 이는 법적으로 문제가 될 수 있습니다.

**사용자 상황에 맞는 조언**:
- **법률 검토**: 계약서 내용을 법률 전문가와 상의하여 현재 조항의 적법성을 확인하세요. 특히 임대인의 책임 범위와 임차인의 의무를 명확히 구분하는 것이 중요합니다.
- **합의 시도**: 임대인과의 직접적인 협의를 통해 합리적인 해결책을 모색하세요. 예를 들어, 일부 비용 분담이나 수리 범위 조정 등이 고려될 수 있습니다.

**위험 경고**:
- **강행규정 위반**: 계약서에 명시된 내용이 법률상 강행규정 위반으로 간주될 경우, 임차인

In [18]:
%%time
# 사용예시 : gpt-4o-mini

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 계약서에 벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다라고 되어있어요

✅ 답변 : 결론: 불가능

📖 법적 근거:
주택임대차보호법 제8조에 따르면, 임차인은 계약 기간 중 정상적인 사용으로 인한 물건의 자연적 마모나 손상에 대해 책임을 지지 않습니다. 또한, 민법 제618조에서는 임대인의 사용과 수익에 대한 권리와 의무를 규정하고 있으며, 임대인은 일반적으로 임차인이 사용 중 생긴 정상적인 손상에 대해 책임지지 않도록 돕는 의무가 있습니다.

사용자 상황에 맞는 조언:
계약서에 “세입자가 전부 수리하고 나간다”는 조항이 포함되어 있는 경우, 이는 법에 저촉될 수 있습니다. 특히, 벽지와 같은 일반적인 마모에 대한 수리에 대해 세입자가 책임을 지는 것은 불리한 특약으로 해석될 수 있으며, 이는 법적으로 인정되지 않을 가능성이 높습니다. 계약서의 해당 조항이 강행규정에 위반된다면, 법원에서 무효로 판단될 수도 있습니다.

🚨 위험 경고: 계약서에 부적절한 특약이 포함된 경우, 세입자에게 불리한 결과를 초래할 수 있으며, 이는 법적으로 불리한 상황을 만들 수 있습니다. 꼭 법률 전문가와 상담하여 계약서의 내용을 조정하거나 검토받는 것이 좋습니다.

❗ 면책: 본 답변은 법률적 조언이 아니며, 법적 효력이 없습니다. 정확한 법률 상담을 위해 변호사와 상담하시기 바랍니다.

📌 메인 근거(priority=1) : 주택임대차보호법
CPU times: total: 875 ms
Wall time: 26.1 s


In [20]:
%%time
# 사용예시 : gpt-4.1-mini

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 임대차계약증서에 벽지 등이 손상될 경우 임차인이 전부 수선의무하고 주택의인도한다고 되어있어요

✅ 답변 : 결론:  
계약서에 ‘벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다’는 특약 조항은 일부 인정되나, 세입자에게 지나치게 과도한 수리 의무를 부과하는 경우(통상적인 사용으로 인한 손상 포함)에는 법적으로 무효 또는 제한될 수 있습니다. 따라서 전부 수리 의무가 무조건적으로 적용된다고 보기 어렵습니다.

📖 법적 근거:  
- **민법 제623조(임대인의 수리의무 등)**: 임대차 목적물의 관리·수리는 원칙적으로 임대인이 부담합니다. 다만 과실이나 고의로 인한 손상에 한해 임차인이 배상할 책임이 있습니다.  
- **주택임대차보호법 제3조(대항력)**: 임차인의 주거 안정 보호 차원에서 통상적인 사용에 따른 손상이나 마모는 임차인 책임으로 보기 어렵습니다.  
- 대법원 판례에 따르면, 통상적인 사용으로 인한 벽지 변색, 노후된 벽지 손상에 대해 세입자에게 전부 수리를 요구하는 것은 부당할 수 있습니다.

사용자 상황에 맞는 조언:  
- 계약서의 ‘전부 수리’ 조항이 통상적인 사용으로 인한 손상까지 포함하여 과도하게 해석될 경우, 임차인에게 불리하게 작용할 수 있으므로 주의해야 합니다.  
- 입주 시 벽지 상태를 상세히 사진 등으로 기록하여 ‘기존 손상’과 ‘임차인 책임 손상’을 구분하세요.  
- 과도한 수리 부담이 예상될 경우 임대인과 협의하거나 법률 상담을 받는 것이 권장됩니다.

🚨 위험 경고:  
- 계약서에 통상적인 마모나 노후에 대한 수리 책임까지 임차인에게 전가하는 것은 **민법과 주택임대차보호법상 강행규정 위반이 될 수 있으므로 매우 위험**합니다.  
- 불리한 특약은 분쟁 시 무효 또는 제한될 가능성이 높으니 계약 전 면밀히 검토하시기 바랍니다.

❗ 면책:  
본 답변은 참고용이며, 구체 상황에 따라 법원의 판단이 달라

In [24]:
%%time
# 사용예시 : bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 계약서에 벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다고 되어있어요.

✅ 답변 : **결론**: 불가능합니다.

**법적 근거**: **주택임대차보호법 제31조**

**해석 및 조언**: 주택임대차보호법 제31조에 따르면, 임대인이 임대차 계약서에서 세입자가 전부 수리하고 나간다는 내용을 명시하는 것은 불법입니다. 이는 강제규정 위반이며, 법적으로 무효로 간주됩니다.

**주의사항**: 이 조항은 임대인과 세입자 모두에게 적용되므로, 해당 내용이 포함된 계약서는 무효로 처리되어야 합니다. 이를 수정하거나 재협의하여 합법적인 내용으로 변경해야 합니다. 만약 이에 대한 불만이 있다면, 전문 변호사와 상담을 통해 법률적 조언을 받으시기 바랍니다.

**주의사항**: 위 내용은 참고 법령에 기반한 분석이며, 실제 법률 적용 시에는 다양한 요소가 고려될 수 있으므로, 상황에 따라 변동이 있을 수 있습니다.
CPU times: total: 641 ms
Wall time: 2min 27s


In [30]:
%%time
# 사용예시 : mervinpraison/llama3.1-instruct:8b

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://developers.upstage.ai/docs/apis/embeddings", 'type': 'invalid_request_error', 'param': 'input', 'code': None}}